# Lecture 4

## The Validation Set Approach

Please run the following cell with `Shift+Enter` to see the video.

In [ ]:
IRdisplay::display_html('<iframe width="640" height="360" src="https://tube.switch.ch/embed/33f91644" frameborder="0" allow="fullscreen"></iframe>')

Let us generate the artificial dataset from the slides with 150 points in total.

In [ ]:
f <- function(x) 0.3 + 2*x - .8*x^2 - .4*x^3
data.generator <- function(N = 150) {
    x <- rnorm(N)
    y <- f(x) + rnorm(N)
    data.frame(X = x, Y = y)
}
set.seed(44)
data <- data.generator()
plot(data)
curve(f, from = -4, to = 4, add = T, col = "orange")

Next we split off a test set that we won't touch until the very end and a
training set that we will later further divide into training and validation set.

In [ ]:
data.test <- data[101:150,]
data <- data[1:100,]

In the next cell we define the function `fit.and.evaluate` to fit a polynomial
model of degree $d$ to the training data and return the training and the
validation error. The function `evaluate.split` computes these errors for a
given split and all degrees from $d = 1$ to $d = 10$.

In [ ]:
fit.and.evaluate <- function(d, train, validat) {
    fit <- lm(Y ~ poly(X, d), train)
    c(mean((train$Y - predict(fit, train))^2),
      mean((validat$Y - predict(fit, validat))^2))
}
evaluate.split <- function(idx.train) {
    d <- seq(1:10)
    sapply(d, fit.and.evaluate, data[idx.train,], data[-idx.train,])
}
set.seed(17)
plot(c(), ylab = "error", xlab = "degree", xlim = c(1, 10), ylim = c(0, 5)) # creates an empty plot
for (i in 1:20) { # fit and evaluate for 20 different splits
    idx.train <- sample(nrow(data), nrow(data)/2)
    res <- evaluate.split(idx.train)
    optimal.d <- which.min(res[2,])  # the which.min function finds the argmin of the vector res[2,]
    lines(res[1,], type = 'l', lwd = .5, col = 'red')
    lines(res[2,], type = 'l', lwd = .5, col = 'blue')
    points(optimal.d, res[2,optimal.d], col = 'blue', pch = 19) # mark with a dot the optimal degree
}
legend("topright", legend = c("training", "validation"), col = c("red", "blue"), lty = 1)

We see that the validation set approach finds different degrees of the
polynomial depending on the split.

In the following two cells we estimate the test error with first a
fit on half of the data and then a fit on all data.
The minimal test error can be found by looking at the data generation process.
What is it's value? Once you know the answer, evaluate the following two cells.

In [ ]:
fit1 <- lm(Y ~ poly(X, 3), data[1:50,])
mean((data.test$Y - predict(fit1, data.test))^2)

In [ ]:
fit2 <- lm(Y ~ poly(X, 3), data)
mean((data.test$Y - predict(fit2, data.test))^2)

You can see that the model fitted on all training data estimates the test error
better than the model fitted on only half the data. (Not too surprisingly :))

If we only care about making as accurate predictions as possible, we should use
all available data to fit the model. In the following cell we combine again the
training set and the test set with the function `rbind`, perform a fit on all
available data and evaluate the new fit together with the fits from the previous
two cells on a really large, newly generated test function to convince ourselves
that it is indeed best to fit on all available data.

In [ ]:
data.all <- rbind(data, data.test)
fit3 <- lm(Y ~ poly(X, 3), data.all)
large.test.data <- data.generator(10^5)
sapply(list(fit1, fit2, fit3), function(fit) mean((large.test.data$Y - predict(fit, large.test.data))^2))

You can now solve the 3 questions on the first page of this week's
[quiz](https://moodle.epfl.ch/mod/quiz/view.php?id=1099910).

## Cross-Validation

In [ ]:
IRdisplay::display_html('<iframe width="640" height="360" src="https://tube.switch.ch/embed/39baffeb" frameborder="0" allow="fullscreen"></iframe>')

### Leave-One-Out Cross Validation
We continue with the example from above. To run cross-validation we define the
function `cv.fit.and.evaluate` that takes as first argument the indices of the
samples to be left out from training (`idx.leftout`); the validation set is
constructed with exactly those samples that are not part of the training set.
In the second to last line we provide every index of the data set once as input to
the `cv.fit.and.evaluate` function to run leave-one-out cross-validation.

In [ ]:
cv.fit.and.evaluate <- function(idx.leftout, data, d = 1) {
    fit.and.evaluate(d, data[-idx.leftout,], data[idx.leftout,])[2]
}
errors <- sapply(1:nrow(data), cv.fit.and.evaluate, data)
head(errors)

Now we can average accross all validation errors we obtained in the previous
cell to compute $\mathrm{CV}_{(n)}$.

In [ ]:
mean(errors)

We repeat now the steps from above for all degrees $d=1,\ldots,10$, plot
$\mathrm{CV}_{(n)}$ and mark with a dot the optimal degree.
We use a so-called anonymous function in the first line; it is anonymous,
because we do not give it a name. If you want to better understand what is
happening here, you are adviced to play around a bit, e.g. what do you think
will be the result of `sapply(1:4, function(x) x^2 + 1)`?

In [ ]:
CV.n <- sapply(1:10, function(d) mean(sapply(1:nrow(data), cv.fit.and.evaluate, data, d)))
plot(CV.n, type = 'l', col = 'blue', ylim = c(0, 5), lwd = 2, xlab = "degree")
optimal.d <- which.min(CV.n)
points(optimal.d, CV.n[optimal.d], col = 'blue', pch = 19)

### k-fold Cross Validation

In the following cell we first define a function that gives as the index sets
for the different folds. In the output you should see that we have 5 times a
list of 20 integers: these are the indices of the samples that are in the
validation set; all other indices will make up the training set.

In [ ]:
CV.k.indices <- function(data, k) {
    n <- floor(nrow(data)/k) # compute the size of each fold
    ordering <- sample(1:nrow(data), nrow(data)) # reshuffle the indices
    lapply(1:k, function(i) ordering[seq((i-1)*n +1, i*n)])
}
idxs <- CV.k.indices(data, 5)
idxs

In the following cell we run 10 times 5-fold cross-validation to get the figure
from the slides.

In [ ]:
plot(c(), xlim = c(1, 10), ylim = c(0, 5), xlab = "degree", ylab = "CV(k)")
for(i in 1:10) {
    idxs <- CV.k.indices(data, 5)
    CV.k <- sapply(1:10, function(d) mean(sapply(idxs, cv.fit.and.evaluate, data, d)))
    lines(CV.k, col = 'blue')
    optimal.d <- which.min(CV.k)
    points(optimal.d, CV.k[optimal.d], col = 'blue', pch = 19)
}

While it may help understanding to code everything oneself, it can also become
annoying to repeat the same steps again and again, when you apply machine
learning in your daily work. Fortunately, there are quite a few nice libraries
that simplify these tasks. One of these libraries that we will start to use more
often now, is the `tidymodels` library. It contains the functions
`initial_split`, `training`, `testing`, `vfold_cv`, `analysis` and `assessment`
that we will use in the cell below to perform the same steps as above in far
fewer lines of code.

In [ ]:
library(tidymodels)
train_test_split <- initial_split(data.all, prop = 2/3) # split data; 2/3 will be training data
data_train <- training(train_test_split) # extract the training set from the split
data_test <- testing(train_test_split)
validation_data <- vfold_cv(data_train, v = 5) # create the 5 folds
tidy_fit_and_evaluate <- function(fold, d) {
    fit <- lm(Y ~ poly(X, d), analysis(fold)) # the function `analysis` extracts the training set from the fold (marked blue in the slides)
    validation_set <- assessment(fold) # the function `assessment` extracts the validation set from the fold (marked blue in the slides)
    mean((validation_set$Y - predict(fit, validation_set))^2)
}
sapply(1:10, function(d) mean(sapply(validation_data$splits, tidy_fit_and_evaluate, d)))

You can now solve the questions on the second page of this week's
[quiz](https://moodle.epfl.ch/mod/quiz/view.php?id=1099910).

## Model Selection with the Akaike Information Criterion

In [ ]:
IRdisplay::display_html('<iframe width="640" height="360" src="https://tube.switch.ch/embed/8ec3bdbc" frameborder="0" allow="fullscreen"></iframe>')

The standard `lm` function does not automatically compute the AIC, but the `glm`
function does. We use it here with the argument `family = "gaussian"` to perform
standard linear least-squares regression and return the AIC.

In [ ]:
fit.and.aic <- function(d, data) {
    fit <- glm(Y ~ poly(X, d), data, family = "gaussian")
    summary(fit)$aic
}
aics <- sapply(1:10, fit.and.aic, data)
plot(aics, type = "l", xlab = "degree", ylab = "AIC")
d <- which.min(aics)
points(d, aics[d])

You can see, that we would have found the optimal degree $d = 3$ also with
the AIC criterion.

You can now solve the questions on the third page of this week's
[quiz](https://moodle.epfl.ch/mod/quiz/view.php?id=1099910).

## Example: Find Relevant Predictors with Cross-Validation

In [ ]:
IRdisplay::display_html('<iframe width="640" height="360" src="https://tube.switch.ch/embed/94316bc2" frameborder="0" allow="fullscreen"></iframe>')

We come back here to the artificial data set with the latent cause from last
week.

In [ ]:
set.seed(19)
N <- 100
z <- runif(N)
data <- data.frame(X1 = 0.7*z, X2 = z + .01*rnorm(N),
                   X3 = runif(N), Y = rep(0, N))
beta <- c(-.8, -.4, 2, -.5)
m <- model.matrix(Y ~ ., data)
data$Y <- m %*% beta + .1 * rnorm(N)

Let us now run all possible models and print the $R^2$ values and the
fitted coefficients. In contrast to the previous section - where we used the AIC to
adjust the training error for different flexibilities of the model - we are only
interested in the ranking of different models with identical flexibility in the
first step of subset selection. This is why we use the $R^2$ values here to find
for each number of predictors the best model.

In [ ]:
fit.and.print <- function(formula, data) {
    fit <- lm(formula, data)
    r.squared <- summary(fit)$r.squared
    print(formula)
    print(r.squared)
    print(coef(fit))
    r.squared
}
fit.subsets <- sapply(c(Y ~ 1,
                        Y ~ X1, Y ~ X2, Y ~ X3,
                        Y ~ X1 + X2, Y ~ X1 + X3, Y ~ X2 + X3,
                        Y ~ .), fit.and.print, data)
fit.subsets

You see that the $R^2$ values for formulas `Y ~ X1` and `Y ~ X2` are very close
to each other. Likewise, the $R^2$ values for formulas `Y ~ X1 + X3` and `Y ~ X2 + X3` are very similar to each other. The highest $R^2$ value, i.e. the lowest
training error has the last model with all predictors.

Let us plot the results as a function of the number of predictors.

In [ ]:
plot(c(0, 1, 1, 1, 2, 2, 2, 3), fit.subsets,
     xlab = "number of predictors",
     ylab = parse(text = "R^2"))

Fortunately, also for this task there is a library to help us. The library
`leaps` contains the function `regsubsets`. The result we see from the following
cell shows which predictors to keep in the best model of a given size. For
example the `*` in the first row of the output indicates that the best model
with one predictor is the one with `X2`, the best with 2 predictors is the one
with `X2` and `X3`. Check, if this is consistent with the results we obtained
above.

In [ ]:
library(leaps)
reg.fit <- regsubsets(Y ~ ., data)
summary(reg.fit)

We can also extract the $R^2$ values for the best performing models.

In [ ]:
summary(reg.fit)$rsq

And the coefficients of the best model with `id = k` predictors.

In [ ]:
coef(reg.fit, id = 1)

Fits obtained with `regsubsets` do unfortunately not have a `predict` function.
We can write our own one instead.

In [ ]:
predict.regsubsets <- function(object, newdata, id, form = as.formula(object$call[[2]])) {
    mat = model.matrix(form, newdata)
    coefi = coef(object, id=id)
    xvars = names(coefi)
    mat[,xvars]%*%coefi
}

While the dot `.` often does not have any specific meaning in R, it does have
one when defining generic functions for different objects. For example
`predict(lm(Y ~ ., data))` actually calls the function `predict.lm`; similarly
there is a `predict.glm` function. You can look at all variants of `predict` with
`methods(predict)`. Thus, when we write `predict(reg.fit, data, 1)` this will
call our new `predict.regsubsets` function.

Let us check, if our new predict function computes the right result.
Because we know that the best model with 2 predictors is the one with predictors
`X2` and `X3`, we compare it to the predictions we obtain with a standard `lm`
fit.

In [ ]:
head(predict(reg.fit, data, 2) - predict(lm(Y ~ X2 + X3, data)))

Let us now perform 10-fold cross-validation to find the best number of predictors.
We use again the functions form the `tidymodels` library.

In [ ]:
validation_data <- vfold_cv(data, v = 10)
fit_and_evaluate <- function(fold, formula = Y ~ .) {
    fit <- regsubsets(formula, analysis(fold))
    valid.set <- assessment(fold)
    sapply(seq(1, fit$nvmax - 1),
           function(id) mean((valid.set$Y - predict(fit, valid.set, id, formula))^2))
}
cv.errors <- sapply(validation_data$splits, fit_and_evaluate)
cv.errors

Let us now average the errors over the 10 fold to obtain the final result.

In [ ]:
rowMeans(cv.errors)

We see that the model with 2 predictors is slightly favoured by
cross-validation. Thus we have indeed found one of the simplified models with
our model selection.

You can now solve the questions on the fourth page of this week's
[quiz](https://moodle.epfl.ch/mod/quiz/view.php?id=1099910).

## Regularization

In [5]:
IRdisplay::display_html('<iframe width="640" height="360" src="https://tube.switch.ch/embed/efa7e9b6" frameborder="0" allow="fullscreen"></iframe>')

### Lasso (L1 regularization)

To perform linear regression with L1 regularization we load the library `glmnet`
and represent the input data as a matrix and the response as a vector.
The `glmnet` function perform linear regression with L1 regularization if
`alpha = 1` and L2 regularization if `alpha = 0`. The `lambda` argument sets the
size of the allowed area. Here we give it a sequence of 100 points between
$10^-2$ and $10$ to perform linear regression with different values of the size
of the allowed area.

In [ ]:
library(glmnet)
x <- as.matrix(data[,c("X1", "X2", "X3")])
y <- data$Y
lasso.mod <- glmnet(x, y, alpha = 1, lambda = 10^seq(1, -2, length = 100))
plot(lasso.mod, "lambda", xlab = parse(text = "Log(lambda)"))

The function `cv.glmnet` performs 10-fold cross-validation to find the optimal
value for the parameter `lambda`.

In [ ]:
cv.lasso <- cv.glmnet(x, y, alpha = 1, nfold = 10)
plot(cv.lasso)

Let us finally run the Lasso with the optimal `lambda`. The coefficients found
are very similar to the ones found with cross-validated subset selection: again
$\beta_1 = 0$ and the other values are pretty close to the ones found with
`coef(lm(Y ~ X2 + X3, data))`; they are just slightly smaller as a result of the
L1 regularization.

In [ ]:
best.lasso <- glmnet(x, y, alpha = 1, lambda = cv.lasso$lambda.min)
coef(best.lasso)

You can now solve the questions on the last page of this week's
[quiz](https://moodle.epfl.ch/mod/quiz/view.php?id=1099910).

## Exercises
### Conceptual
**Q1.** We formulated regularization as a constrained optimization problem with one
inequality constraint.  A standard approach to solve constraint optimization
problems makes use of Karush-Kuhn-Tucker (KKT) multipliers. For example, to find
the minimum of function $f(x)$ under the constraint $g(x) \leq s$ one can define
the loss function $$L(x, \lambda) = f(x) + \lambda(g(x) - s)$$ where
$\lambda\geq0$ is a KKT multiplier.  Minimizing the loss both in $x$ and
$\lambda$ amounts to solving the equations $\frac{\partial L}{\partial x} = f'(x)
+ \lambda g'(x) = 0$ and $\frac{\partial L}{\partial \lambda} = g(x) - s = 0$,
if the solution is on the boundary of the area defined by the inequality
constraint; otherwise one can find the solution by simply solving the
unconstrained problem, i.e. with $\lambda = 0$. In this formulation one choses
the size $s$ of the allowed area and find $\lambda$ by solving the equations.

Interestingly, the loss function $$L(x) = f(x) + \lambda
g(x)$$ has exactly the same partial derivative in $x$ as $L(x, \lambda)$ and
therefore all critical points of $L(x)$ have corresponding critical
points of $L(x,\lambda)$. Because of this, regularization is often formulated
as "adding a regularization term to the cost function". For example, given loss
function of linear regression $L(\beta) = \mathrm{RSS}$ one can define the
L1-regularized loss function $L_1(\beta) = \mathrm{RSS} + \lambda
\|\beta\|_1$ and choose a value for $\lambda$ instead of the size $s$ of the
allowed area.

(a) Argue, why choosing $\lambda = 0$ in the second formulation is equivalent to
choosing $s = \infty$ in the first formulation.

(b) Argue, why choosing $\lambda = \infty$ in the second formulation is
equivalent to choosing $s = 0$ in the first formulation.

**Q2.** Consider a data set with as many data points as predictors $n = p$.
Assume $x_{ii} = 1$ and $x_{ij} = 0$ for all $i\neq j$ and arbitrary values
$y_i$. To simplify the problem further we perform regression without an
intercept. We would like to study L1- and L2-regularized multiple linear regression.

(a) Write the RSS loss once with L1 regularization and once with L2
regularization for this setting and the second fomulation of regularization seen
in Q1.

(b) Show that in the case of L2 regularization the estimated coefficients take
the form $\hat \beta_j = y_j/(1 + \lambda)$.

(c) Show that in the case of L1 regularization the estimated coefficients take
the form $\hat \beta_j = y_j - \lambda/2$, if $y_j > \lambda/2$,
$\hat \beta_j = y_j + \lambda/2$, if $y_j < -\lambda/2$ and $\hat \beta_j = 0$
otherwise.

(d) Write a brief summary on how the estimated coefficients $\hat \beta_j$ are
changed relative to the unregularized solution for both kinds of regularization.

(e) Compare your solutions with the text in section "A Simple Special Case
for Ridge Regression and the Lasso" of the text book (page 224).

**Q3.** We now review k-fold cross-validation.

(a) Explain how k-fold cross-validation is implemented.

(b) What are the advantages and disadvantages of k-fold cross-validation
relative to:

i. The validation set approach?

ii. LOOCV?

### Applied

**Q4.** Create an artificial dataset with 10 points, 4 predictors $X_1, X_2, X_3, X_4$
and $Y = X_1 + \epsilon$ with $\mathrm{Var}(\epsilon) = 0.1^2$.

(a) Run subset selection and report all $R^2$ values.

(b) Which model has the lowest training error?

(c) Find with cross-validation the model which has the lowest expected test error.

(d) Find with cross-validation and the lasso the best model.

**Q5.** In this exercise we work again with the life expectancy dataset.

(a) Find with cross-validation the best degree $d$ of polynomial regression with
the single predictor GDP and response LifeExpectancy.

(b) Find with cross-validated subset selection the best subset of predictors in
multiple linear regression with predictors GDP, BMI, Year and Alcohol and
response LifeExpectancy.

(c) Run L1 and L2 regularized multiple linear regression on the full data set
and plot the coefficients as a function of $\lambda$ for both types of
regulization.

(d) Run the same task as in (b) with cross-validated L1 regularization.

(e) Adapt the subset selection algorithm from the slides to choose among the
models $\mathcal M_k$ with the AIC criterion instead of cross-validation and run
the same task as in (b) with AIC as the model comparison method.

(optional) R has quite a few built-in data sets. You can get a list of the data
sets with the function `data()`. Pick one data set you like. As a running
example I will use the "swiss" data set, but you can pick another one. Load the
data set, e.g. with `data("swiss")`. You can get more information about the data
set with `?swiss`. Analyse the data set you picked with the machine learning
methods you know already.